# LABORATORIO 5: Códigos de las Señales EEG con BITalino

In [ ]:
#Importamos las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, welch

#### Funciones para filtrado de EEG
Se definen las funciones de acuerdo al rango necesario para filtrar las señales de ECG, usualmente hasta un paso de banda de 0.5 a 40 Hz.

In [ ]:
# Filtros
def filtro_pasabanda(senal, fs, frec_baja=0.5, frec_alta=40.0, orden=4):
    nyquist = 0.5 * fs
    bajo = frec_baja / nyquist
    alto = frec_alta / nyquist
    b, a = butter(orden, [bajo, alto], btype="band")
    return filtfilt(b, a, senal)

def filtro_notch(senal, fs, frec_notch=60.0, Q=30.0):
    nyquist = 0.5 * fs
    w0 = frec_notch / nyquist
    b, a = iirnotch(w0, Q)
    return filtfilt(b, a, senal)

## 1. Reposo
### 1.1 Preprocesado y acondicionamiento para reposo

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("basal.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

### 1.2 Señal cruda

In [ ]:
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 10)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 750000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 1000)
plt.show()

### 1.3 Señal filtrada

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0,10)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 220000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 1000)
plt.show()


## 2. Mirada  fija
### 2.1 Preprocesado y acondicionamiento para mirada fija (toma 1)

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("Fijar_mirada_1.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

### 2.2 Señal Cruda (toma 1)

In [ ]:
# Ploteos
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 16)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 750000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 5000)
plt.show()

### 2.3 Señal Filtrada (toma 1)

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 16)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 750000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 5000)
plt.show()

### 2.4 Preprocesado y acondicionamiento para mirada fija (toma 2)

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("Fijar_mirada_2.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

### 2.5 Señal Cruda (toma 2)

In [ ]:
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 15)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 300000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()

### 2.6 Señal filtrada (toma 2)

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 15)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 300000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()

## 3. Mirada sin luz
### 3.1 Preprocesado y acondicionamiento para mirada sin luz (toma 1)

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("Mirada_apagada_1.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]


### 3.2 Señal cruda (toma 1)

In [ ]:
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 15)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 500000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()

### 3.3 Señal filtrada (toma 1)

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 5)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 420000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()


### 3.4 Preprocesado y acondicionamiento para mirada sin luz (toma 2)

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("Mirada_apagada_2.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

### 3.5 Señal cruda (toma 2)

In [ ]:
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 15)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 520000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()

### 3.6 Señal filtrada (toma 2)

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 15)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 500000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()



## 4. Parpadeo cada 2 segundos
### 4.1 Preprocesado y acondicionamiento para parpadeo cada 2 seg (toma 1)

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("parpadeo1.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

### 4.2 Señal Cruda (toma 1)

In [ ]:
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 850000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 5000)
plt.show()

### 4.3. Señal Filtrada (toma 1)

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 850000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 5000)
plt.show()


### 4.4 Preprocesado y acondicionamiento para parpadeo cada 2 seg (toma 2)

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("parpadeo1.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

### 4.5 Señal Cruda (toma 2)

In [ ]:
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 850000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 4000)
plt.show()

### 4.6 Señal filtrada (toma 2)

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 850000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 4000)
plt.show()



## 5. Resta desde 100 a 0 
### 5.1 Preprocesado y acondicionamiento para resta

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("resta1.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

### 5.2 Señal Cruda

In [ ]:
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 15)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 500000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()

### 5.3 Señal filtrada

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 5)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 420000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()


## 6. Escuchar música
### 6.1 Señal cruda y filtrada (Ondas Alfa)

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("ondas_alfa.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

# Ploteos
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 650000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 4000)
plt.show()

#SEÑAL FILTRADA 
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 650000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 4000)
plt.show()


### 6.2 Señal cruda y filtrada (Ondas Beta)

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("ondas_beta.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

# Ploteos
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 650000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 4000)
plt.show()

#SEÑAL FILTRADA 
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 650000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 4000)
plt.show()


### 6.3 Señal cruda y filtrada (canción libre)

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("musica.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

# Ploteos
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 650000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()

#SEÑAL FILTRADA 
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 650000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 3000)
plt.show()


## 7. Preguntas 
### 7.1 Señal cruda y filtrada 

In [ ]:
# Cargamos los datos del archivo .txt de OpenSignals
datos = np.loadtxt("preguntas.txt", delimiter=None, comments="#")
eeg = datos[:, 5] #La señal se encuentra en ña 5 columna

fs = 1000  
t = np.arange(len(eeg)) / fs

#Señal filtrada
eeg_filtrado = filtro_pasabanda(eeg, fs, 0.5, 40)
eeg_filtrado = filtro_notch(eeg_filtrado, fs, 60)

#FFT
fft_vals_cruda = np.fft.rfft(eeg)
fft_freqs_cruda = np.fft.rfftfreq(len(eeg), 1/fs)
fft_db_cruda = 20*np.log10(np.maximum(np.abs(fft_vals_cruda),1e-12)/np.max(np.abs(fft_vals_cruda)))

# FFT filtrada
fft_vals_filt = np.fft.rfft(eeg_filtrado)
fft_freqs_filt = np.fft.rfftfreq(len(eeg_filtrado), 1/fs)
fft_db_filt = 20*np.log10(np.maximum(np.abs(fft_vals_filt),1e-12)/np.max(np.abs(fft_vals_filt)))

#PSD (Welch)
f_cruda, psd_cruda = welch(eeg, fs=fs, nperseg=fs*2)
f_filt, psd_filt = welch(eeg_filtrado, fs=fs, nperseg=fs*2)

bandas = {"Delta (0.5–4 Hz)": (0.5, 4),"Theta (4–8 Hz)": (4, 8),"Alfa (8–12 Hz)": (8, 12),"Beta (13–30 Hz)": (13, 30),"Gamma (30–40 Hz)": (30, 40)}

potencias_cruda = []
for low, high in bandas.values():
    idx = np.logical_and(f_cruda >= low, f_cruda <= high)
    potencia = np.trapezoid(psd_cruda[idx], f_cruda[idx])
    potencias_cruda.append(potencia)

potencias_filt = []
for low, high in bandas.values():
    idx = np.logical_and(f_filt >= low, f_filt <= high)
    potencia = np.trapezoid(psd_filt[idx], f_filt[idx])
    potencias_filt.append(potencia)


colores = ["#FF6F61","#6B5B95","#88B04B","#FFA500","#00BFFF"]

# Ploteos
plt.figure(figsize=(12, 10))
#SEÑAL CRUDA
plt.subplot(4,1,1)
plt.plot(t, eeg, color="darkblue")
plt.title("EEG cruda")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_cruda, np.abs(fft_vals_cruda), color="darkorange")
plt.title("FFT - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 850000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_cruda, fft_db_cruda, color="darkgreen")
plt.title("FFT en dB - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#Welch
plt.subplot(4,1,4)
plt.semilogy(f_cruda, psd_cruda, color="blue")
plt.title("PSD (Welch) - EEG cruda")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_cruda, color=colores)
plt.title("Potencia por banda - EEG cruda")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 6000)
plt.show()

#SEÑAL FILTRADA 
plt.figure(figsize=(12, 10))

plt.subplot(4,1,1)
plt.plot(t, eeg_filtrado, color="darkgreen")
plt.title("EEG filtrada (0.5–40 Hz + notch 60 Hz)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud (uV)")
plt.xlim(0, 40)
plt.grid()
#FFT
plt.subplot(4,1,2)
plt.plot(fft_freqs_filt, np.abs(fft_vals_filt), color="darkred")
plt.title("FFT - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.xlim(0, 100)
plt.ylim(0, 850000)
plt.grid()
#FFT en dB
plt.subplot(4,1,3)
plt.plot(fft_freqs_filt, fft_db_filt, color="purple")
plt.title("FFT en dB - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud (dB)")
plt.xlim(0, 100)
plt.grid()
#WELCH
plt.subplot(4,1,4)
plt.semilogy(f_filt, psd_filt, color="red")
plt.title("PSD (Welch) - EEG filtrada")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("PSD (uV^2/Hz)")
plt.xlim(0, 50)
plt.grid()

plt.tight_layout()
plt.show()

#PSD 
plt.figure(figsize=(8,4))
plt.bar(bandas.keys(), potencias_filt, color=colores)
plt.title("Potencia por banda - EEG filtrada")
plt.ylabel("Potencia (uV^2)")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.ylim(0, 6000)
plt.show()
